# Comparative Subject Wi2Vi

In [1]:
# Imports
# !pip install torchinfo
import torch
import torch.nn as nn
import torch.utils.data as Data
from torchvision import datasets, transforms
from torchinfo import summary
import matplotlib.pyplot as plt
import os
# os.environ['OPENBLAS_NUM_THREADS'] = '1'
import numpy as np
import gc
import random
import time
import CompMethods_V3 as TP
import DataSetting_v5 as DS

In [2]:
gpu = 0
date = '20240917'
name = f'{date}_COMP_sub'
data_path = ['../dataset/Door_EXP/A308']
level = 'subject'

data_organizer = DS.DataOrganizer(name, data_path, level)
data_organizer.load()

Cross validation plan at subject level
Loading ../dataset/Door_EXP/A308...

Loaded 0716D00-csi.npy of shape (1172731, 30, 3)
Loaded 0716D00-pd.npy of shape (1172731, 62)
Loaded 0716D01-csi.npy of shape (999414, 30, 3)
Loaded 0716D01-pd.npy of shape (999414, 62)
Loaded 0716D02-csi.npy of shape (1249977, 30, 3)
Loaded 0716D02-pd.npy of shape (1249977, 62)
Loaded 0716D03-csi.npy of shape (1022668, 30, 3)
Loaded 0716D03-pd.npy of shape (1022668, 62)
Loaded 0717D04-csi.npy of shape (976630, 30, 3)
Loaded 0717D04-pd.npy of shape (976630, 62)
Loaded 0717D05-csi.npy of shape (1055272, 30, 3)
Loaded 0717D05-pd.npy of shape (1055272, 62)
Loaded 0717D10-csi.npy of shape (10727, 30, 3, 3)
Loaded 0717D11-csi.npy of shape (10562, 30, 3, 3)
Loaded 0717D12-csi.npy of shape (8569, 30, 3, 3)
Loaded 0717D13-csi.npy of shape (10707, 30, 3, 3)
Loaded 20240716_145635-bbx.npy of shape (6080, 4)
Loaded 20240716_145635-cimg.npy of shape (6080, 1, 128, 128)
Loaded 20240716_145635-ctr.npy of shape (6080, 2)
Load

### Wi2Vi

In [3]:
preprocess_wi2vi = DS.Preprocess(new_size=(240, 320))
data_organizer.load_plan('../dataset/Door_EXP/subject_r0.2_A308.pkl')

def run():
    for i in range(6):
        data_organizer.gen_plan()
        train_loader, valid_loader, test_loader, current_test = data_organizer.gen_loaders(mode='s', num_workers=1)
    
        wi2vi = TP.Wi2Vi()
        Wi2Vi_trainer = TP.CompTrainer(name='Wi2Vi', mask=False,
                                      networks=[wi2vi],
                                       lr=1e-4, epochs=10, cuda=gpu,
                                       preprocess = preprocess_wi2vi,
                                       loss_optimizer = {'LOSS': [torch.optim.Adam, 1e-4]},
                                       train_loader=train_loader, valid_loader=valid_loader, test_loader=test_loader, 
                                       notion=f'{name}_{current_test}_Wi2Vi')
        wi2vi_trained = Wi2Vi_trainer.schedule()

        del Wi2Vi_trainer
        del wi2vi_trained
        torch.cuda.empty_cache()
        gc.collect()